In [1]:
# standard imports and setup
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
get_ipython().run_line_magic('matplotlib', 'inline')
from sklearn.model_selection import train_test_split
from imblearn.datasets import make_imbalance
import re
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

In [2]:
tweets_df = pd.read_csv("labeled_data.csv")
pd.set_option('display.max_rows', None)
tweets_df = tweets_df.drop(columns = ['count', 'hate_speech', 'offensive_language', 'neither'])
tweets_df['class'].value_counts()

1    19190
2     4163
0     1430
Name: class, dtype: int64

In [3]:
tweets_df['class'] = tweets_df['class'].replace(to_replace = 1, value = 0)
tweets_df['class'] = tweets_df['class'].replace(to_replace = 2, value = 1)
tweets_df.head()

,Unnamed: 0,class,tweet
0,0,1,!!! RT @mayasolovely: As a woman you shouldn't...
1,1,0,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...
2,2,0,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...
3,3,0,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...
4,4,0,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...


In [4]:
tweets_df['class'].value_counts()

0    20620
1     4163
Name: class, dtype: int64

In [5]:
# Separate Data
X = tweets_df.drop(['class'],axis=1)
y = tweets_df['class']
X_train, X_test, y_train, y_test = train_test_split(X,y)

# Data balancing
X_train_1 = tweets_df.drop(['class'],axis=1)
y_train_1 = tweets_df['class']
X_train, y_train = make_imbalance(X_train_1, y_train_1, sampling_strategy={0: 4163, 1: 4163},random_state=14)

In [6]:
len(X_train) #8326
len(y_train) #8326

8326

In [7]:
tweets_df = pd.DataFrame()
tweets_df.insert(0, 'class', y_train)
tweets_df.insert(1, 'tweet', X_train['tweet'])
tweets_df.to_csv('dataPreprocessing.csv', index=False)
tweets_df.head()

,class,tweet
0,0,RT @joytooawesome: @gorgeouss_kee @vintage_mon...
1,0,CNT nobody be mad at who he choose to be with ...
2,0,I'll be a bitch if I have to. I don't like to ...
3,0,@kieffer_jason ha bitch who angry bitch am tal...
4,0,@Dre_thinks lol lil bitch


In [8]:
import re
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
# Process data

def data_processing(tweet):
    #tweet=re.sub("[&#]",'', tweet, flags = re.MULTILINE)
    tweet=re.sub(":",'',tweet)
    tweet=re.sub(";",'',tweet)
    
    #remove Digits
    tweet=re.sub("(\s)\d+(\s)", "\1\2", tweet)
    #remove Links
    tweet=re.sub('http[s]?://\S+', '', tweet)
    
    #remove @user
    tweet=re.sub(r'(\s)?@\w+', r'\1',tweet)
    #remove Emojis
    tweet=re.sub(r'(\s)?&#\w+', r'\1',tweet)
    tweet_tokens = word_tokenize(tweet)
    filtered_tweets = [w for w in tweet_tokens if not w in stop_words]
    return " ".join(filtered_tweets)

In [9]:
tweets_df.tweet = tweets_df['tweet'].apply(data_processing)
tweets_df.to_csv('dataPreprocessingFilter.csv', index=False)